In [2]:
import yfinance as yf
import pandas as pd
import datetime

# Define the list of tickers for the selected equities
tickers = ['CRWD', 'NVDA', 'HD']

# Initialize an empty dictionary to store the data
data = {}

# Retrieve 2-minute interval data for the past 60 days
start_date = (datetime.datetime.now() - datetime.timedelta(days=59)).strftime('%Y-%m-%d')
end_date = datetime.datetime.now().strftime('%Y-%m-%d')

for ticker in tickers:
    print(f"Fetching data for {ticker}")
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date, interval='2m')
        data[ticker] = stock_data
        
        # Save each stock's data into a separate CSV file
        stock_data.to_csv(f"{ticker}_2min_data.csv")
        
    except Exception as e:
        print(f"Could not retrieve data for {ticker}: {e}")

print("Data retrieval and saving complete!")

Fetching data for CRWD


[*********************100%%**********************]  1 of 1 completed


Fetching data for NVDA


[*********************100%%**********************]  1 of 1 completed


Fetching data for HD


[*********************100%%**********************]  1 of 1 completed

Data retrieval and saving complete!


In [6]:
# Load the previously saved CRWD data
crwd_data = pd.read_csv("CRWD_2min_data.csv", index_col=0, parse_dates=True)

# Check the first few rows of the data to ensure it loaded correctly
print("Initial data check:")
print(crwd_data.head())

# Calculate the 20-minute and 200-minute SMAs
crwd_data['20min_SMA'] = crwd_data['Close'].rolling(window=10).mean()  # 20min SMA on a 2min interval is a 10-period SMA
crwd_data['200min_SMA'] = crwd_data['Close'].rolling(window=100).mean()  # 200min SMA on a 2min interval is a 100-period SMA

# Check if the SMA columns are populated correctly
if crwd_data['20min_SMA'].isnull().all() or crwd_data['200min_SMA'].isnull().all():
    print("Error: SMA columns are not populated correctly. Please check the calculations.")
else:
    print("SMA columns populated successfully. Sample data:")
    print(crwd_data[['20min_SMA', '200min_SMA']].dropna().head())

# Save the updated data with SMAs to the same CSV file or a new one
crwd_data.to_csv("CRWD_2min_data_with_SMA.csv")

print("Process completed successfully! 20min & 200min SMA data added to CSV file!")

Initial data check:
                                 Open        High         Low       Close  \
Datetime                                                                    
2024-07-08 09:30:00-04:00  390.368011  397.038086  390.279999  395.589996   
2024-07-08 09:32:00-04:00  395.704987  396.899994  394.220001  394.220001   
2024-07-08 09:34:00-04:00  394.100006  394.334991  383.670013  386.410004   
2024-07-08 09:36:00-04:00  385.619995  385.619995  379.790009  382.119995   
2024-07-08 09:38:00-04:00  382.434998  385.459900  381.490997  385.230011   

                            Adj Close  Volume  
Datetime                                       
2024-07-08 09:30:00-04:00  395.589996  241134  
2024-07-08 09:32:00-04:00  394.220001   95122  
2024-07-08 09:34:00-04:00  386.410004  176206  
2024-07-08 09:36:00-04:00  382.119995  178854  
2024-07-08 09:38:00-04:00  385.230011   80863  
SMA columns populated successfully. Sample data:
                            20min_SMA  200min_SMA
Datet

In [7]:
# Now add the Setup condition
crwd_data['Setup'] = crwd_data['20min_SMA'] > crwd_data['200min_SMA']

# Save the updated data with the Setup column to the same or new CSV file
crwd_data.to_csv("CRWD_2min_data_with_SMA_and_Setup.csv")

# Re-load the final file to confirm everything saved correctly
final_data = pd.read_csv("CRWD_2min_data_with_SMA_and_Setup.csv", index_col=0, parse_dates=True)

# Check if the SMA and Setup columns are populated in the final CSV
if final_data[['20min_SMA', '200min_SMA', 'Setup']].isnull().all().any():
    print("Error: One or more columns (SMA, Setup) are not populated correctly in the final CSV.")
else:
    print("Final CSV file saved correctly. Sample data:")
    print(final_data[['20min_SMA', '200min_SMA', 'Setup']].dropna().head())

print("Process completed successfully! First portion of setup with identifying Picture of Power is complete!")

Final CSV file saved correctly. Sample data:
                            20min_SMA  200min_SMA  Setup
Datetime                                                
2024-07-08 12:48:00-04:00  387.350009  387.061303   True
2024-07-08 12:50:00-04:00  387.349509  386.981053   True
2024-07-08 12:52:00-04:00  387.392209  386.917353   True
2024-07-08 12:54:00-04:00  387.464410  386.932753   True
2024-07-08 12:56:00-04:00  387.566412  386.992253   True
Process completed successfully!


In [5]:
# Load the CRWD data with SMAs and Setup
crwd_data = pd.read_csv("CRWD_2min_data_with_SMA_and_Setup.csv", index_col=0, parse_dates=True)

# Initialize all bars as "Noise Bar" by default
crwd_data['Label'] = "Noise Bar"

# Identify Control Bars
crwd_data.loc[(crwd_data['Close'] > crwd_data['Open']) & (crwd_data['Setup'] == True), 'Label'] = "Control Bar"

# Iterate through the DataFrame to label Ignored Bars and Indicator Bars
for i in range(1, len(crwd_data) - 1):
    if crwd_data.iloc[i]['Label'] == "Control Bar":
        # We have identified a Control Bar; now check subsequent bars

        # Check for Ignored Bar A
        if (crwd_data.iloc[i + 1]['Close'] < crwd_data.iloc[i + 1]['Open']) and \
           ((crwd_data.iloc[i + 1]['Open'] - crwd_data.iloc[i + 1]['Close']) <= 0.5 * (crwd_data.iloc[i]['Close'] - crwd_data.iloc[i]['Open'])):
            crwd_data.iloc[i + 1, crwd_data.columns.get_loc('Label')] = "Ignored Bar A"

            # Check for Ignored Bar B
            if (i + 2 < len(crwd_data)) and \
               (crwd_data.iloc[i + 2]['Close'] < crwd_data.iloc[i + 2]['Open']) and \
               ((crwd_data.iloc[i + 2]['Open'] - crwd_data.iloc[i + 2]['Close']) <= 0.5 * (crwd_data.iloc[i]['Close'] - crwd_data.iloc[i]['Open'])):
                crwd_data.iloc[i + 2, crwd_data.columns.get_loc('Label')] = "Ignored Bar B"
                next_index = i + 2
            else:
                next_index = i + 1
            
            # Check for Indicator Bar
            if (next_index + 1 < len(crwd_data)) and \
               (crwd_data.iloc[next_index + 1]['Close'] > crwd_data.iloc[next_index + 1]['Open']) and \
               (crwd_data.iloc[next_index + 1]['High'] > crwd_data.iloc[next_index]['High']):
                crwd_data.iloc[next_index + 1, crwd_data.columns.get_loc('Label')] = "Indicator Bar"

# Final Review: Relabel any "Control Bar" that is not followed by an "Ignored Bar" as a "Noise Bar"
for i in range(len(crwd_data) - 1):
    if crwd_data.iloc[i]['Label'] == "Control Bar":
        if crwd_data.iloc[i + 1]['Label'] not in ["Ignored Bar A", "Ignored Bar B"]:
            crwd_data.iloc[i, crwd_data.columns.get_loc('Label')] = "Noise Bar"

# Save the labeled data to a new CSV file
crwd_data.to_csv("CRWD_2min_data_with_Final_Labels.csv")

print("Candlestick labeling completed and saved successfully!")


Candlestick labeling completed and saved successfully!


In [6]:
# Load the CRWD data with final labels
crwd_data = pd.read_csv("CRWD_2min_data_with_Final_Labels.csv", index_col=0, parse_dates=True)

# Calculate the slope (first derivative) of the 20-minute SMA
crwd_data['SMA_20_slope'] = crwd_data['20min_SMA'].diff()

# Initialize all bars as "Empty"
crwd_data['Trade_Status'] = "Empty"

# Check the conditions after identifying Ignored Bars
for i in range(len(crwd_data) - 1):
    if crwd_data.iloc[i]['Label'] in ["Ignored Bar A", "Ignored Bar B"]:
        # Condition 1: Check if the closing price is within $1.00 of the 20-minute SMA
        proximity_condition = abs(crwd_data.iloc[i]['Close'] - crwd_data.iloc[i]['20min_SMA']) <= 1.00
        
        # Condition 2: Check if the slope of the 20-minute SMA is positive
        slope_condition = crwd_data.iloc[i]['SMA_20_slope'] > 0
        
        # If both conditions are true, label the bar as "Primed"
        if proximity_condition and slope_condition:
            crwd_data.iloc[i, crwd_data.columns.get_loc('Trade_Status')] = "Primed"

# Save the updated data with the Trade_Status column to a new CSV file
crwd_data.to_csv("CRWD_2min_data_with_Trade_Status.csv")

print("Proximity and slope checks completed, and labels applied successfully!")


Proximity and slope checks completed, and labels applied successfully!


In [20]:
# Load the CRWD data with the latest labels
crwd_data = pd.read_csv("CRWD_2min_data_with_Trade_Status.csv", index_col=0, parse_dates=True)

# Initialize a DataFrame to record trades
trade_log = pd.DataFrame(columns=["Entry_Time", "Entry_Price", "Stop_Loss", "Trade_Type"])

# Create a list to track active positions
active_positions = []

# Iterate through the DataFrame to identify and execute trades
for i in range(1, len(crwd_data) - 1):
    current_price = crwd_data.iloc[i]['Close']
    current_sma = crwd_data.iloc[i]['20min_SMA']
    
    # Check for each active position
    for position in active_positions[:]:  # Use a copy of the list to modify it during iteration
        entry_time, entry_price, stop_loss, trade_type = position
        
        # Condition A: Exit if the price moves $1.00 or more away from the 20-minute SMA
        if abs(current_price - current_sma) >= 1.00:
            # Log the sell due to moving away from SMA
            new_trade = pd.DataFrame({
                "Entry_Time": [crwd_data.index[i]],
                "Entry_Price": [current_price],
                "Stop_Loss": [stop_loss],
                "Trade_Type": ["Sell - Away from SMA"]
            })
            trade_log = pd.concat([trade_log, new_trade], ignore_index=True)
            
            # Remove the position from active_positions
            active_positions.remove(position)
            print(f"Position closed due to moving away from SMA at {crwd_data.index[i]}: Sell Price = {current_price}")

        # Condition B: Exit if the stop loss is triggered
        elif current_price <= stop_loss:
            # Log the sell due to stop loss
            new_trade = pd.DataFrame({
                "Entry_Time": [crwd_data.index[i]],
                "Entry_Price": [current_price],
                "Stop_Loss": [stop_loss],
                "Trade_Type": ["Sell - Stop Loss"]
            })
            trade_log = pd.concat([trade_log, new_trade], ignore_index=True)
            
            # Remove the position from active_positions
            active_positions.remove(position)
            print(f"Position closed due to stop loss at {crwd_data.index[i]}: Sell Price = {current_price}")
    
    # Check if a new position was opened (from the previous logic)
    if crwd_data.iloc[i]['Label'] == "Indicator Bar" and \
       crwd_data.iloc[i]['Close'] > crwd_data.iloc[i-1]['High'] + 0.01:
        # Record the new buy position
        new_position = (
            crwd_data.index[i],  # Entry time
            crwd_data.iloc[i-1]['High'] + 0.01,  # Entry price
            crwd_data.iloc[i-1]['Low'],  # Stop loss
            "Buy"  # Trade type
        )
        active_positions.append(new_position)
        
        # Log the buy
        new_trade = pd.DataFrame({
            "Entry_Time": [crwd_data.index[i]],
            "Entry_Price": [crwd_data.iloc[i-1]['High'] + 0.01],
            "Stop_Loss": [crwd_data.iloc[i-1]['Low']],
            "Trade_Type": ["Buy"]
        })
        trade_log = pd.concat([trade_log, new_trade], ignore_index=True)

# Save the trade log to a new CSV file
trade_log.to_csv("CRWD_Trade_Log.csv", index=False)

print("Trade identification and logging completed successfully!")

/var/folders/2n/rmtzbdn93h3f2tkgn0yypj300000gn/T/ipykernel_22630/2179825284.py:68: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trade_log = pd.concat([trade_log, new_trade], ignore_index=True)


Position closed due to stop loss at 2024-07-08 13:54:00-04:00: Sell Price = 387.635009765625
Position closed due to moving away from SMA at 2024-07-08 15:54:00-04:00: Sell Price = 390.9100036621094
Position closed due to stop loss at 2024-07-10 14:18:00-04:00: Sell Price = 372.2099914550781
Position closed due to moving away from SMA at 2024-07-10 15:34:00-04:00: Sell Price = 373.7000122070313
Position closed due to stop loss at 2024-07-12 10:58:00-04:00: Sell Price = 372.1265869140625
Position closed due to moving away from SMA at 2024-07-12 11:58:00-04:00: Sell Price = 368.5549926757813
Position closed due to stop loss at 2024-07-12 15:30:00-04:00: Sell Price = 371.5950012207031
Position closed due to moving away from SMA at 2024-07-15 09:58:00-04:00: Sell Price = 378.0249938964844
Position closed due to moving away from SMA at 2024-07-15 10:24:00-04:00: Sell Price = 380.2699890136719
Position closed due to moving away from SMA at 2024-07-15 10:54:00-04:00: Sell Price = 382.820007324

In [21]:
# Load the existing trade log
trade_log = pd.read_csv("CRWD_Trade_Log.csv", parse_dates=["Entry_Time"])

# Sort the trade log by Entry_Time in ascending order
trade_log = trade_log.sort_values(by="Entry_Time", ascending=True)

# Round all monetary values to the nearest hundredth and format to keep two decimal places
trade_log["Entry_Price"] = trade_log["Entry_Price"].round(2).apply(lambda x: f"{x:.2f}")
trade_log["Stop_Loss"] = trade_log["Stop_Loss"].round(2).apply(lambda x: f"{x:.2f}")

# Save the updated and sorted trade log
trade_log.to_csv("CRWD_Trade_Log.csv", index=False)

print("Trade log sorted, rounded, and formatted with two decimal places successfully!")

Trade log sorted, rounded, and formatted with two decimal places successfully!
